# QuPath for Python Programmers 🐍
**Alan O'Callaghan, Léo Leplat, Peter Bankhead, Fiona Inglis, Laura Nicolás Sáenz**

## Introduction/motivation

QuPath is user-friendly for analysing large images interactively

Python is great for developing and implementing image analysis algorithms with code

They don't *always* cooperate nicely... but this is changing.

## Paquo

Paquo is an existing library for QuPath-python
interactions.

*but* it provides a somewhat "heavy-weight" linkage between the two.
This is great when you want a well-designed program that uses both languages.

QuBaLab aims to be a bit more light-weight, and doesn't aim to provide
direct linkage of Java and Python code to the same extent.

This means it's good for short experiments, but really not
great for complex applications!

In [ ]:
cache_folder = "./i2k-qupath-python-project/images"

# Define a utility function to find or download an image

from pathlib import Path
import urllib.request

if cache_folder != "":
    Path(cache_folder).mkdir(parents=True, exist_ok=True)

def get_image(image_name, image_url):
    if cache_folder == "":
        filename = None
    else:
        filename = Path(cache_folder) / image_name
    
    if filename is None or not(filename.exists()):
        print(f"Downloading {image_name}...")
        path, _ = urllib.request.urlretrieve(image_url, filename=filename)
        print(f'{image_name} saved to {path}')
    else:
        path = filename
        print(f'{image_name} found in {path}')

    return path

In [ ]:
# Download or get image
cmu_path = get_image("CMU-1.svs", "https://openslide.cs.cmu.edu/download/openslide-testdata/Aperio/CMU-1.svs")

In [ ]:
# Download or get image
fluoro_path = get_image("patient_test_1.ome.tiff", "https://ftp.ebi.ac.uk/biostudies/fire/S-BIAD/463/S-BIAD463/Files/my_submission/Validation_raw/DCIS/Patient_test_1.ome.tiff")

## ImageServer

QuBaLab provides a number of ImageServers, that allow you to read images
just as QuPath would.


OpenSlide for RGB images:

In [ ]:
from qubalab.images.openslide_server import OpenSlideServer

print(cmu_path)
openslide_server = OpenSlideServer(cmu_path)

AICSImageIO for general purpose images:

In [ ]:
from qubalab.images.aicsimageio_server import AICSImageIoServer

print(fluoro_path)
aicsimageio_server = AICSImageIoServer(fluoro_path)

ICC profile servers that wrap other servers:

In [ ]:
from qubalab.images.icc_profile_server import IccProfileServer

icc_profile_server = IccProfileServer(openslide_server)

Since we're using an RGB image, we'll use OpenSlide.

In [ ]:
server = openslide_server

## ImageServer operations

In [ ]:
metadata = server.metadata

print(f'Image name: {metadata.name}')
print('Levels:')
for level, shape in enumerate(metadata.shapes):
    print(f'Shape of level {level}: {shape}')
print('Pixel calibration:')
print(f'Pixel length on x-axis: {metadata.pixel_calibration.length_x}')
print(f'Pixel length on y-axis: {metadata.pixel_calibration.length_y}\n')
print(f'Pixel type: {metadata.dtype}')
print(f'Downsamples: {metadata.downsamples}\n')
print('Channels:')
for channel in metadata.channels:
    print(channel)

## Reading images

In [ ]:
highest_downsample = server.metadata.downsamples[-1]
lowest_resolution = server.read_region(highest_downsample)

print(f'Image shape: {lowest_resolution.shape}')

from qubalab.display.plot import plotImage
import matplotlib.pyplot as plt
_, ax = plt.subplots()
plotImage(ax, lowest_resolution)

## Reading regions

In [ ]:
downsample = 1
x = 13000
y = 15000
width = 2000
height = 1000
tile = server.read_region(downsample, x=x, y=y, width=width, height=height)

print(f'Tile shape: {tile.shape}')

_, ax = plt.subplots()
plotImage(ax, tile)

## Reading images lazily

In [ ]:
last_level = server.metadata.n_resolutions - 1
lowest_resolution = server.level_to_dask(last_level)

# Pixel values are not read yet, but you can get the shape of the image
print(f'Image shape: {lowest_resolution.shape}')

# Compute array. This will read the pixel values
lowest_resolution = lowest_resolution.compute()

_, ax = plt.subplots()
plotImage(ax, lowest_resolution)

## Reading images lazily

In [ ]:
highest_resolution = server.level_to_dask(0)
print(f'Full resolution image shape: {highest_resolution.shape}')

x = 13000
y = 15000
width = 2000
height = 1000
tile = highest_resolution[:, y:y+height, x:x+width]
print(f'Tile shape: {tile.shape}')

tile = tile.compute() #  This will only read the pixel values of the tile
_, ax = plt.subplots()
plotImage(ax, tile)

## Setting up a gateway

The main method of interacting with QuPath
is through a *gateway*, which operates using
a websocket connection:

In [ ]:
from qubalab.qupath import qupath_gateway

token = None
port = 25333
gateway = qupath_gateway.create_gateway(auth_token=token, port=port)

gateway

## QuPath setup

We're using a [CC0 TMA image](https://dataverse.harvard.edu/file.xhtml?persistentId=doi:10.7910/DVN/GG0D7G/VCRA28&version=1.0).


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(qupath_gateway.create_snapshot())
plt.axis(False)
plt.show()

## Basic gateway operations

A gateway provides us with an *entry point*,
which allows us to call Java methods from python:


In [ ]:
print(f"Extension version: {gateway.entry_point.getExtensionVersion()}")

In [ ]:
print(f"Current image name: {gateway.entry_point.getCurrentImageName()}")

## Mixing 🐍 and ☕

Since the entry point exposes the QuPath scripting interface, you can do lots of basic scripting operations in python by calling object methods.

In [ ]:
cores = gateway.entry_point.getTMACoreList()

tumor_cores = [core for core in cores if core.getClassification() == "Tumor"]

[(core.getName(), core.getChildObjects().size()) for core in tumor_cores]

## Downsides

However, it's not ideal to write Java code in python, and we wouldn't necessarily encourage it.

If you just want to script QuPath, groovy will
remain the best bet.

## Objects

In [ ]:
from qubalab.objects.object_type import ObjectType

object_type = ObjectType.ANNOTATION    # could be DETECTION, TILE, CELL, TMA_CORE

In [ ]:
annotations = qupath_gateway.get_objects(object_type = object_type)
annotation = annotations[0]

annotation.setName("Hello from Python")

In [ ]:
qupath_gateway.refresh_qupath()

plt.imshow(qupath_gateway.create_snapshot())
plt.axis(False)
plt.show()

## Converting objects

In [ ]:
annotations = qupath_gateway.get_objects(object_type = object_type, converter='geojson')

print(type(annotations[0]))

In [ ]:
from shapely.geometry import shape

shape(annotations[0].geometry)

## Adding and deleting objects

In [ ]:
import random

for annotation in annotations:
    annotation.color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

qupath_gateway.add_objects(annotations)

## Building a workflow using ImageServer and python

Reading the image currently open

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qubalab.images.qupath_server import QuPathServer
from skimage.filters import gaussian, threshold_otsu
from skimage.color import rgb2gray
from qubalab.objects.image_feature import ImageFeature

qupath_server = QuPathServer(gateway) # use image currently opened in QuPath
downsample = 20 # reduce size by 20x
image = qupath_server.read_region(downsample=downsample)

## Building a workflow using ImageServer and python

Converting to grayscale and filtering

In [ ]:
# If the image is RGB, we convert it to grayscale
# read_region() returns an image with the (c, y, x) shape.
# To use rgb2gray, we need to move the channel axis so that
# the shape becomes (y, x, c)
image = np.moveaxis(image, 0, -1)
image = rgb2gray(image)

# Apply a gaussian filter
image = gaussian(image, 2.0)

## Building a workflow
- Identify the threshold
- Apply the threshold to make a mask
- Convert the mask to an image feature
- Import the image feature as an annotation
- 

In [ ]:
threshold = threshold_otsu(image)

mask = image < threshold

mask_annotation = ImageFeature.create_from_label_image(
    mask,   
    scale=downsample,   # mask is 20 times smaller than the QuPath image, so we scale
                        # the annotations to fit the QuPath image
    classification_names="Otsu",  # set a single classification to the detected annotations
)
qupath_gateway.add_objects(mask_annotation)

In [ ]:
plt.imshow(mask)
plt.title(f'Otsu (threshold={threshold:.2f})')
plt.axis(False)
plt.show()

## Other workflow ideas

Object classifiers using scikit-learn

Object clustering using graph clustering

Assessing classifier feature importance
